# Locally Deployment of a Registered Model for Debugging Purposes

The same code of the previous notebook is used here, except the deployment target (local Docker).

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model

In [ ]:
import azureml.core
print(azureml.core.VERSION)

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
from azureml.core.model import Model

model = Model(ws, 'aml-wrkshp-classif-empl-attrition')

print('Model name: ', model.name, '\n', 'Model description: ', model.description, '\n', 'Model version: ', model.version, sep='')

In [ ]:
# Download the model locally in order to test the score.py script
model.download(target_dir='.', exist_ok=True)

In [ ]:
%%writefile score.py

import os
import json
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
#from sklearn.externals import joblib
import joblib

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame(data=[{'Age': 49, 'BusinessTravel': 'Travel_Rarely', 'DailyRate': 1098, 'Department': 'Research & Development', 'DistanceFromHome': 4, 'Education': 2, 'EducationField': 'Medical', 'EnvironmentSatisfaction': 4, 'Gender': 'Female', 'HourlyRate': 21, 'JobInvolvement': 3, 'JobLevel': 2, 'JobRole': 'Laboratory Technician', 'JobSatisfaction': 3, 'MaritalStatus': 'Single', 'MonthlyIncome': 711, 'MonthlyRate': 2124, 'NumCompaniesWorked': 8, 'OverTime': 1, 'PercentSalaryHike': 8, 'PerformanceRating': 4, 'RelationshipSatisfaction': 3, 'StockOptionLevel': 0, 'TotalWorkingYears': 2, 'TrainingTimesLastYear': 0, 'WorkLifeBalance': 3, 'YearsAtCompany': 2, 'YearsInCurrentRole': 1, 'YearsSinceLastPromotion': 0, 'YearsWithCurrManager': 1}])
output_sample = np.array([0])


def init():
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION).
    # If there are multiple models, this value is the path to the directory containing all deployed models (./azureml-models).
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'classif-empl-attrition.pkl')
    
    # Deserialize the model file back into a sklearn model
    model = joblib.load(model_path)


@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

In [ ]:
import sklearn
from azureml.core.environment import Environment

environment = Environment("LocalDeploy")

environment.python.conda_dependencies.add_conda_package("python=3.6.2")
environment.python.conda_dependencies.add_conda_package("pandas")
environment.python.conda_dependencies.add_conda_package("numpy")

environment.python.conda_dependencies.add_pip_package("azureml-core==1.17.0")
environment.python.conda_dependencies.add_pip_package("azureml-defaults==1.17.0")
environment.python.conda_dependencies.add_pip_package("sklearn-pandas")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn==0.22.2.post1")

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=environment)

In [ ]:
from azureml.core.webservice import LocalWebservice

webservice_name = 'predict-attrition-test'

# Giving a specific port is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, webservice_name, [model], inference_config, deployment_config)

In [ ]:
local_service.wait_for_deployment()

In [ ]:
print('Local service port: {}'.format(local_service.state))
print('Local service port: {}'.format(local_service.port))

In [ ]:
print(local_service.get_logs())